In [1]:
import pandas as pd
import time

In [2]:
df = pd.read_csv('../../data/ml/filtered.csv', parse_dates=['fecha'], low_memory=False)

In [3]:
df.head()

,fecha,idema,tmed,prec,tmin,tmax,hr_max,hr_media,latitud,altitud,fecha_sin,fecha_cos
0,2010-01-01,2331,2.7,3.5,0.8,4.6,97.0,94.0,42.3500,891,0.5086,0.9999
1,2010-01-01,0320I,3.4,0.0,0.5,6.4,95.0,57.0,42.3167,1151,0.5086,0.9999
2,2010-01-01,C428T,16.2,0.0,13.7,18.8,77.0,69.0,28.1667,418,0.5086,0.9999
3,2010-01-01,C239N,17.1,0.0,12.0,22.2,86.0,69.0,28.2000,1,0.5086,0.9999
4,2010-01-01,B158X,13.0,0.0,11.2,14.8,70.0,59.0,39.5500,50,0.5086,0.9999


In [4]:
nan_percentage = df.isna().mean() * 100
nan_percentage = nan_percentage[nan_percentage > 0].sort_values(ascending=False)
nan_percentage.apply(lambda x: f"{x:.2f}%")

hr_max      5.90%
hr_media    5.60%
prec        3.64%
tmed        2.27%
tmin        2.24%
tmax        2.23%
dtype: object

In [5]:
lookup = pd.read_csv('../../data/locations/estacion_distance_lookup.csv')

In [6]:
df_imputed = df.copy()

In [7]:
def impute_knn(col, k=3):

    # Iterate through each day
    for fecha in df_imputed['fecha'].unique():
        current_row = df_imputed[df_imputed['fecha'] == fecha]

        # Filter NaN and non NaN rows
        nan_rows = current_row[current_row[col].isna()]
        non_nan_rows = current_row[current_row[col].notna()]

        for index, row in nan_rows.iterrows():
            
            # Use the nearest k stations to take the average
            nearest_idemas = lookup[row['idema']]
            nearest_values = non_nan_rows[non_nan_rows['idema'].isin(nearest_idemas)][col][:k]

            if not nearest_values.empty:
                imputed_value = nearest_values.mean()
                df_imputed.loc[index, col] = imputed_value

Since the datasets will be split by year, this does not introduce any data leakage as it goes day by day

In [8]:
start = time.time()
for col in ['tmed', 'tmax', 'tmin', 'prec', 'hr_media', 'hr_max']:
    col_start = time.time()
    impute_knn(col)
    elapsed = time.time() - col_start
    print(f"Imputed {col} in {elapsed}s")
print(f"Imputed all columns in {time.time() - start}s")

Imputed tmed in 73.81400847434998s
Imputed tmax in 72.14162540435791s
Imputed tmin in 72.4633777141571s
Imputed prec in 92.14604640007019s
Imputed hr_media in 120.82074546813965s
Imputed hr_max in 126.15736556053162s
Imputed all columns in 557.5436522960663s


In [9]:
df_imputed.head()

,fecha,idema,tmed,prec,tmin,tmax,hr_max,hr_media,latitud,altitud,fecha_sin,fecha_cos
0,2010-01-01,2331,2.7,3.5,0.8,4.6,97.0,94.0,42.3500,891,0.5086,0.9999
1,2010-01-01,0320I,3.4,0.0,0.5,6.4,95.0,57.0,42.3167,1151,0.5086,0.9999
2,2010-01-01,C428T,16.2,0.0,13.7,18.8,77.0,69.0,28.1667,418,0.5086,0.9999
3,2010-01-01,C239N,17.1,0.0,12.0,22.2,86.0,69.0,28.2000,1,0.5086,0.9999
4,2010-01-01,B158X,13.0,0.0,11.2,14.8,70.0,59.0,39.5500,50,0.5086,0.9999


In [13]:
df_imputed.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4582780 entries, 0 to 4582779
Data columns (total 12 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   fecha      4582780 non-null  datetime64[ns]
 1   idema      4582780 non-null  object        
 2   tmed       4582780 non-null  float64       
 3   prec       4582780 non-null  float64       
 4   tmin       4582780 non-null  float64       
 5   tmax       4582780 non-null  float64       
 6   hr_max     4582780 non-null  float64       
 7   hr_media   4582780 non-null  float64       
 8   latitud    4582780 non-null  float64       
 9   altitud    4582780 non-null  int64         
 10  fecha_sin  4582780 non-null  float64       
 11  fecha_cos  4582780 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 419.6+ MB


In [14]:
df_imputed.to_csv('../../data/ml/full.csv', index=False)